# Integrantes:

Andres Santiago Jiménez Guzmán \
Jhasley Nathalia Pinzón Saiz

El proyecto que se muestra a continuación, se realiza con el fin de generar un código de clasificación, el cual a partir de una serie de características obtenidas por los patrones de las imágenes del artículo de turbidez (https://iwaponline.com/wst/article/84/10-11/2749/82733/The-effect-of-mobile-camera-selection-on-the) se busca generar una clasificación multiclase; para esto, se necesitó de la obtención previa de algunas características en las imágenes, en este caso se definieron el promedio de los contornos, del área de los contornos y la detección de esquinas de Harris, luego el paso a seguir es realizar la partición en dos conjuntos de datos más pequeños que serán utilizadas con los siguientes fines: entrenamiento, validación y test. Para el caso de la validación, se utilizaron varios modelos los cuales permitieron verificar que tan eficiente era el algoritmo en cada caso y así elegir el mejor modelo para aplicar en el test.

# Librerias

Las librerías implementadas permiten:

-pandas: La manipulación y el análisis de datos \
-os: Acceso a directorios (para leer y escribir archivos) \
-cv2: Tratamiento de las imágenes \
-numpy: Para el cálculo de vectores y matrices con funciones matemáticas de alto nivel 




In [7]:
import pandas as pd
import os
import cv2 as cv
import numpy as np


# Funciones

La función image_processing(),se encarga del procesamiento de cada imagen de la base de datos de imágenes que se encuentra en el directorio "DB", por lo que para el tratamiento y obtención de característica, se redimensiona la imagen para así obtener tres características en este caso un promedio de los bordes detectados, un promedio de áreas de los contornos y el promedio de la detección de ángulos de Harris, lo cual se va guardando en los diferentes arreglos correspondientes a cada una de ellas, para posteriormente en la función excel_file() crear un archivo en el que se almacena la información de cada imagen generada previamente.

In [8]:
def image_processing(dir):
    for file_name in dir:
        images_dir='DB/Train/'+file_name
        images = os.listdir(images_dir)
        for c in images:
            if file_name=="Circle":
                figure_label.append(1)
            elif file_name=="Squares":
                figure_label.append(2)
            else:
                figure_label.append(3)

            figure.append(file_name)
            image_path = images_dir+'/'+c
            src = cv.imread(image_path)
            dsize = (200, 50)
            output = cv.resize(src, dsize)
            img_gray = cv.cvtColor(output, cv.COLOR_BGR2GRAY)
            img_blur = cv.GaussianBlur(img_gray, (5,5), 0)
            
            #Corners Harris
            dst = cv.cornerHarris(img_blur,2,3,0.04)
            dst = cv.dilate(dst,None)
            cornersharris.append(np.sum(dst.flatten())/len(dst))
            
            #Contornos y Area de contorno
            canny = cv.Canny(img_blur, 50, 150)
            (contornos,_) = cv.findContours(canny.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE )
            suma = []
            area=[]
            
            #Promedio de los contornos y áreas
            for c in contornos:
                epsilon = 0.02*cv.arcLength(c,True)
                approx = cv.approxPolyDP(c,epsilon,True)
                suma.append(np.sum(approx)/len(approx))
                area.append(cv.contourArea(c))
            bordersprom.append(np.sum(suma)/len(contornos))
            areaborders.append(np.sum(area)/len(area))
            suma = []
            area=[]


def excel_file():
    data = {'Figure_label':figure_label,'Figure': figure,'Borders_prom': bordersprom,'Areas_prom':areaborders,'Angle_Harris': cornersharris}
    df = pd.DataFrame(data, columns=['Figure_label', 'Figure', 'Borders_prom','Areas_prom','Angle_Harris'])
    df.to_excel('data/datosObtenidos.xlsx', sheet_name='Data')


A continuación en el main se hace la lectura de todas las imágenes que se encuentran en el directorio "Train" de las cuales se obtienen los datos anteriormente mencionados y se guardan en un archivo Excel por medio de las funciones image_processing() y excel_file()

In [9]:
def main():
    ejemplo_dir = 'DB/Train'
    files_names = os.listdir(ejemplo_dir)
    image_processing(files_names)
    excel_file()

Finalmente, se configuran los arreglos en los cuales se almacenan las características para que se puedan enviar a la base de datos correctamente

In [10]:
ID=[]
figure_label=[]
figure=[]
bordersprom=[]
areaborders=[]
cornersharris=[]

if __name__=="__main__":
    main()


Ahora se continua con el archivo models.ipynb